In [1]:
import ipyvuetify as v
import ipywidgets
import csv

In [2]:
#input list
file_input = None
file_name = None
country_selection = None
drawing_method = None

In [3]:
#output list 
asset_name = None
year = None

In [4]:
#map widget
import geemap

center = [0, 0]
zoom = 1

dc = geemap.DrawControl(marker={},
                 circlemarker={},
                 polyline={},
                 rectangle={'shapeOptions': {'color': '#0000FF'}},
                 circle={'shapeOptions': {'color': '#0000FF'}},
                 polygon={'shapeOptions': {'color': '#0000FF'}},
                 )

m = geemap.Map(center=center, zoom=zoom)
dark_matter_layer = geemap.basemap_to_tiles(geemap.basemaps.CartoDB.DarkMatter)
m.clear_layers()
m.clear_controls()
m.add_layer(dark_matter_layer)
m.add_control(geemap.ZoomControl(position='topright'))
#m.add_control(dc)
#m

In [5]:
#import file
widget_file_input = v.FileInput(label='select a file', v_model=None, class_="d-none")
def on_file_input_change(widget, event, data):
    global file_input
    file_input = widget.v_model

widget_file_input.on_event('change', on_file_input_change )
#widget_file_input

widget_file_name = v.TextField(label='select a filename', v_model=None, class_="d-none")
def on_file_name_change(widget, event, data):
    global file_name
    file_name = widget.v_model

widget_file_name.on_event('change', on_file_name_change )
#widget_file_name

In [6]:
#country boundaries selector 
def create_FIPS_dic():
    """create the list of the country code in the FIPS norm using the CSV file provided in utils"""
    fips_dic = {}
    with open('./utils/FIPS_code_to_country.csv', newline='') as f:
        reader = csv.reader(f, delimiter=';')
        for row in reader:
            fips_dic[row[1]] = row[0]
        
    return fips_dic

widget_country_selection = v.Select(items=[*create_FIPS_dic()], label='Country/Province', v_model=None, class_="d-none")
def on_country_change(widget, event, data):
    global country_selection
    country_selection = widget.v_model

widget_country_selection.on_event('change', on_country_change )
#widget_country_selection

In [7]:
#select the year of interest 
years = [2020, 2019]
widget_select_year = v.Select(items=years, label='Select a year', v_model=None)

def on_year_change(widget, event, data):
    global year
    year = widget.v_model

widget_select_year.on_event('change', on_year_change )
#widget_select_year

In [8]:
#GEE asset
widget_asset_name = v.TextField(label='select a GEE asset', class_="d-none", v_model=None)

def on_asset_change(widget, event, data):
    global asset_name
    asset_name = widget.v_model
    
widget_asset_name.on_event('change', on_asset_change)

#widget_asset_name

In [9]:
# add a method selector 
selection_method = ['Country boundaries', 'draw a shape', 'Upload file', 'use GEE asset']
#disableling the upload file at the moment 
widget_drawing_method = v.Select(items=selection_method, label='Select an AOI type', v_model=None)

def on_method_change(widget, event, data):
    
    global selection_method
    global widget_file_input
    global widget_file_name
    global widget_country_selection
    global widget_asset_name
    global drawing_method
    
    drawing_method = widget.v_model
    
    if drawing_method == selection_method[0]: 
        widget_file_input.class_= 'd-none'
        widget_file_name.class_='d-none'
        widget_country_selection.class_='d-inline'
        widget_asset_name.class_='d-none'
        m.remove_control(dc)
        dc.clear()
    elif drawing_method == selection_method[1]:
        widget_file_input.class_= 'd-none'
        widget_file_name.class_='d-inline'
        widget_country_selection.class_='d-none'
        widget_asset_name.class_='d-none'
        m.add_control(dc)
    elif drawing_method == selection_method[2]:
        widget_file_input.class_= 'd-inline'
        widget_file_name.class_='d-none'
        widget_country_selection.class_='d-none'
        widget_asset_name.class_='d-none'
        m.remove_control(dc)
        dc.clear()
    elif drawing_method == selection_method[3]:
        widget_file_input.class_= 'd-none'
        widget_file_name.class_='d-none'
        widget_country_selection.class_='d-none'
        widget_asset_name.class_='d-inline'
        m.remove_control(dc)
        dc.clear()
        
    
    

widget_drawing_method.on_event('change', on_method_change)
#widget_drawing_method

In [10]:
widget_alert = v.Alert(children=[''], type='info', text=True, class_="mt-5")
#widget_alert

In [11]:
#validate the selected data 
btn = v.Btn(
    color='primary', 
    children=[
        v.Icon(left=True, children=['mdi-map-marker-check']),
        'Select these inputs'
    ]
)
widget_validate_data = v.Container(children=[btn])


def toggleLoading():
    global btn 
    btn.loading = not btn.loading
    btn.disabled = btn.loading

def on_click(widget, event, data):
    
    global widget_alert
    global drawing_method
    global asset_name
    global m
    
    #io variables 
    global file_input
    global file_name
    global country_selection
    global drawing_method 
    global asset_name
    global year
    
    #check for the year 
    if year == None: 
        widget_alert.type = 'error'
        widget_alert.children = ['You have forgotten to select a year']
        return 
    
    #do something 
    toggleLoading()
    alert_type, message, asset_name = run_GLAD_input(
        file_input, 
        file_name, 
        country_selection, 
        asset_name, 
        drawing_method
    )
    toggleLoading()
    widget_alert.type = alert_type
    widget_alert.children = [message]
    
    
    #update the map with asset
    m.centerObject(ee.FeatureCollection(asset_name), zoom=4)
    m.addLayer(ee.FeatureCollection(asset_name), {'color': 'green'})
    dc.clear()
    m.remove_control(dc)

widget_validate_data.on_event('click', on_click)
#widget_validate_data

In [12]:
#create the display 
inputs = v.Layout(
    _metadata={'mount-id': 'data-input'},
    class_="pa-5",
    row=True,
    align_center=True, 
    children=[
        v.Flex(xs12=True, children=[widget_drawing_method]),
        v.Flex(xs12=True, children=[widget_country_selection]),
        v.Flex(xs12=True, children=[widget_file_input]),
        v.Flex(xs12=True, children=[widget_file_name]),
        v.Flex(xs12=True, children=[widget_asset_name]),
        v.Flex(xs12=True, children=[widget_select_year]),
        v.Flex(xs12=True, children=[widget_validate_data]),
        v.Flex(xs12=True, children=[widget_alert]),
    ]
)
#inputs

AOI_content_main =  v.Layout(
    _metadata={'mount_id': 'step-1-widget'},
    xs12=True,
    row=True,
    class_="ma-5",
    children=[
        v.Card(
            class_="pa-5",
            raised=True,
            xs12=True,
            children=[
                v.Html(xs12=True, tag='h2', children=['Select an AOI']),
                v.Layout(
                    row=True,
                    xs12=True,
                    children=[
                        v.Flex(xs12=True, md6=True, children=[inputs]),
                        v.Flex(class_="pa-5", xs12=True, md6=True, children=[m])
                    ]
                )    
            ]
        )
    ]
)
AOI_content_main

Layout(children=[Card(children=[Html(children=['Select an AOI'], tag='h2'), Layout(children=[Flex(children=[La…

In [13]:
import ee
import time

#initialize earth engine
ee.Initialize()

def run_GLAD_input(file_input, file_name, country_selection, asset_name, drawing_method):
    
    global widget_alert
    
    #go to the glad folder 
    folder = ee.data.getAssetRoots()[0]['id'] + '/'
    
    #check the drawing method
    if drawing_method == None:
        alert_type = 'warning'
        message = 'No selection method have bee picked up'
        asset = None
    elif drawing_method == 'Country boundaries':
        if country_selection == None:
            alert_type = 'warning'
            message = 'No Country have been selected'
            asset = None
        else:
            exist = False
            asset_descripsion = 'Glad_' + country_selection.replace(' ', '_') + '_boundaries'
            liste = ee.data.listAssets({'parent': folder})['assets']
            for asset in liste:
                if asset['name'] == folder + asset_descripsion:
                    exist = True
                    break
            #si il existe 
            if exist:
                asset = folder + asset_descripsion
                alert_type = 'info'
                message = "The asset was already existing you can continue to use it. It's also available at :" + folder + asset_descripsion
            else:
                country_code = create_FIPS_dic()[country_selection]
                #create the asset 
                country = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('country_co', country_code))
                
                task_config = {
                    'collection': country, 
                    'description':asset_descripsion,
                    'assetId': folder + asset_descripsion
                }
    
                task = ee.batch.Export.table.toAsset(**task_config)
                task.start()
            
                state = task.state
                while state != 'COMPLETED':
                    widget_alert.children = [str(state)]
                    time.sleep(5)
                    
                    #search for the task in task_list
                    tasks_list = ee.batch.Task.list()
                    current_task = ''
                    for task in tasks_list:
                        if task.config['description'] == asset_descripsion:
                            current_task = task
                            break
                    state = current_task.state
                    
                asset = folder + asset_descripsion
                alert_type = 'success'
                #message = "The asset has been created under the name : " + folder + asset_descripsion
                message = task.state
                
    elif drawing_method == 'draw a shape':
        #create the feature collection 
        aoi = ee.FeatureCollection(m.draw_features)
        asset_name = "Glad_" + file_name.replace(' ', '_')
        #verify the asset name 
        exist = False
        liste = ee.data.listAssets({'parent': folder})['assets']
        for asset in liste:
            if asset['name'] == folder + asset_name:
                exist = True
                break
        if exist:
            asset = None
            alert_type = 'error'
            message = "The name was already in used, change it or delete the previous asset in your GEE acount"
        else:
            task_config = {
                'collection': country, 
                'description':asset_name,
                'assetId': folder + asset_name
            }
            task = ee.batch.Export.table.toAsset(**task_config)
            task.start()
            
            state = task.state
            while state != 'COMPLETED':
                widget_alert.children = [str(state)]
                time.sleep(5)
                    
                #search for the task in task_list
                tasks_list = ee.batch.Task.list()
                current_task = ''
                for task in tasks_list:
                    if task.config['description'] == asset_descripsion:
                        current_task = task
                        break
                state = current_task.state
                    
            asset = folder + asset_name
            alert_type = 'success'
            message = "The drawing asset has been added under the name : " + folder + asset_name
            
    elif drawing_method == 'use GEE asset':
        #verify that there is an asset
        if asset_name == '' or asset_name == None:
            asset = asset_name
            alert_type = 'warning'
            message = "No Asset have been provided"
        else:
            asset = asset_name
            alert_type = 'info'
            message = "Check carefuly that your input is an asset"
    elif drawing_method == 'Upload file':
        ee_object = geemap.shp_to_ee(file_input)
        asset_name = "Glad_" + file_name.replace(' ', '_')
        #verify the asset name 
        exist = False
        liste = ee.data.listAssets({'parent': folder})['assets']
        for asset in liste:
            if asset['name'] == folder + asset_name:
                exist = True
                break
        if exist:
            asset = None
            alert_type = 'error'
            message = "The name was already in used, change it or delete the previous asset in your GEE acount"
        else:
            task_config = {
                'collection': country, 
                'description':asset_name,
                'assetId': folder + asset_name
            }
            task = ee.batch.Export.table.toAsset(**task_config)
            task.start()
            asset = folder + asset_name
            alert_type = 'success'
            message = "The shp file has been added to your GEE asets under the name : " + folder + asset_name
        asset = None
        alert_type = 'error'
        message = "This function is not yet available"
            
    return (alert_type, message, asset)   